In [ ]:
%%capture
!pip install requirements.txt

In [122]:
import json

from PyPDFForm import PdfWrapper

pdf = PdfWrapper("i-140.pdf")

with open('c.json', 'r') as file:
    company_info = json.load(file)

customer_info = {
    "prt2PetitionType[1]": True,
    "Pt3Line1a_FamilyName[0]": "CUSTOMER_FIRST_NAME",
    "Pt3Line1b_GivenName[0]": "CUSTOMER_MIDDLE_NAME",
    "Pt3Line1c_MiddleName[0]": "CUSTOMER_LAST_NAME",
    "Line2a_InCareofName[0]": "CUSTOMER_NAME",
    "Line2b_StreetNumberName[0]": "CUSTOMER_STREET_ADDR",
    "Line2c_Unit[0]": False,
    "Line2c_Unit[1]": False,
    "Line2c_Unit[2]": False,
    "Line2c_AptSteFlrNumber[0]": "C_APT",
    "Line2d_CityOrTown[0]": "CUSTOMER_CITY_OR_TOWN",
    "Line2h_Province[0]": "CUSTOMER_PROVINCE",
    "Line2i_Country[0]": "CUSTOMER_COUNTRY",
    "Line2e_State[0]": 0,
    "Line2f_ZipCode[0]": "C_ZIP",
    "Line2g_PostalCode[0]": "CUSTOMER_POSTAL_CODE",
}

reversed_dict = {v: [k] for k, v in customer_info.items()}

def reverse_dict(d: dict):
    return { in v for k, v in d.items()}

print(reversed_dict)
print(reverse_dict(reversed_dict))

schema = pdf.schema["properties"]

# print(json.dumps(dict(list(schema.items())[50:]), indent=4))
# print(type(schema))

info = company_info | customer_info

pdf = pdf.fill(info)

with open("output.pdf", "wb+") as output_file:
    output_file.write(pdf.read())

{True: ['prt2PetitionType[1]'], 'CUSTOMER_FIRST_NAME': ['Pt3Line1a_FamilyName[0]'], 'CUSTOMER_MIDDLE_NAME': ['Pt3Line1b_GivenName[0]'], 'CUSTOMER_LAST_NAME': ['Pt3Line1c_MiddleName[0]'], 'CUSTOMER_NAME': ['Line2a_InCareofName[0]'], 'CUSTOMER_STREET_ADDR': ['Line2b_StreetNumberName[0]'], False: ['Line2e_State[0]'], 'C_APT': ['Line2c_AptSteFlrNumber[0]'], 'CUSTOMER_CITY_OR_TOWN': ['Line2d_CityOrTown[0]'], 'CUSTOMER_PROVINCE': ['Line2h_Province[0]'], 'CUSTOMER_COUNTRY': ['Line2i_Country[0]'], 'C_ZIP': ['Line2f_ZipCode[0]'], 'CUSTOMER_POSTAL_CODE': ['Line2g_PostalCode[0]']}


NameError: name 'v' is not defined

In [ ]:
from typing import Annotated, Optional
from typing_extensions import TypedDict

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, BaseMessage, HumanMessage, AIMessage

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch

from pydantic import BaseModel, Field

from dotenv import load_dotenv

load_dotenv()

# Keys
OPENROUTER_API_KEY = os.environ.get("OPENROUTER_API_KEY")
TAVILY_API_KEY = os.environ.get("TAVILY_API_KEY")

class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages] # Be explicit about list[BaseMessage]

evidence_string = """
    Evidence of one of the following (each field should be unique): A receipt of lesser nationally or internationally recognized prizes or awards for excellence in the field of endeavor, 
        membership in associations in the field, which require outstanding achievements as judged by recognized national or international experts;
        published material about the alien in professional or major trade publications or other major media,
        participation on a panel or individually as a judge of others’ work in the field or a related field,
        original scientific, scholarly, artistic, athletic, or business-related contributions of major significance in the field,
        authorship of scholarly articles in the field in professional or major trade publications or other major media,
        display of the alien’s work at artistic exhibitions or showcases,
        evidence that the alien has performed in a leading or critical role for organizations or establishments that have distinguished reputations,
        evidence that the alien has commanded a high salary or other high compensation for services,
        evidence of commercial successes in the performing arts as shown by box office receipts or music or video sales.
"""

class ResponseFormatter(BaseModel):
    """Always use this tool to structure your response to the user."""
    achievement_evidence: str = Field(description="Evidence of a one-time achievement (for example, a major internationally recognized award)")
    achievement_evidence_url: str = Field(description="URL for the 'achievement_evidence'")
    field_one: str = Field(description=evidence_string)
    field_one_url: str = Field(description="URL for the 'field_one'")
    field_two: str = Field(description=evidence_string)
    field_two_url: str = Field(description="URL for the 'field_two'")
    field_three: str = Field(description=evidence_string)
    field_three_url: str = Field(description="URL for the 'field_three'")

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="""
            You are an immigration paralegal that does not give legal advice. 
            You are looking for the following information on the subject provided to you by the user:

            1. Evidence of a one-time achievement (for example, a major internationally recognized award); or
            2. At least three of the following:
                a. Receipt of lesser nationally or internationally recognized prizes or awards for excellence in the field of endeavor;
                b. Membership in associations in the field, which require outstanding achievements as judged by recognized national or international experts;
                c. Published material about the alien in professional or major trade publications or other major media;
                d. Participation on a panel or individually as a judge of others’ work in the field or a related field;
                e. Original scientific, scholarly, artistic, athletic, or business-related contributions of major significance in the field;
                f. Authorship of scholarly articles in the field in professional or major trade publications or other major media;
                g. Display of the alien’s work at artistic exhibitions or showcases;
                h. Evidence that the alien has performed in a leading or critical role for organizations or establishments that have distinguished reputations;
                i. Evidence that the alien has commanded a high salary or other high compensation for services; and
                j. Evidence of commercial successes in the performing arts as shown by box office receipts or music or video sales.
        """),
        ("placeholder", "{messages}")
    ]
)


graph_builder = StateGraph(State)

with open('fields.json', 'r') as f:
    site_data = json.load(f)
    sites = [field["domains"] for field in site_data["journals"]]
    sites = [item for sublist in sites for item in sublist]
    sites.append(site_data["additional_general_academic_resources"])
    sites.append(site_data["general_news_sources"])

tavily = TavilySearch(api_key=TAVILY_API_KEY, included_domains=sites)
tools = [tavily]

llm = ChatOpenAI(
    model_name="deepseek/deepseek-chat-v3-0324:free",
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=OPENROUTER_API_KEY
)

llm_with_tools = llm.bind_tools(tools).with_structured_output(ResponseFormatter)

def chatbot(state: State):
    # Invoke the LLM to get the structured output
    formatted_messages = prompt.format_messages(messages=state["messages"])
    structured_output = llm_with_tools.invoke(formatted_messages)

    # Decide how to represent this in the chat history
    # You could create an AIMessage from it, or just store it separately.
    # For simplicity, let's create an AIMessage with a string representation
    ai_message = AIMessage(content=f"Structured Response: {structured_output.model_dump_json(indent=2)}")

    return {
        "messages": [ai_message], # Add the AI message to history
        "structured_response": structured_output # Store the structured object separately
    }

# In your stream_graph_updates, you might then check for 'structured_response'
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [HumanMessage(content=user_input)]}): # Use HumanMessage directly
        for value in event.values():
            if "messages" in value and value["messages"]:
                print("Assistant:", value["messages"][-1].content)
            if "structured_response" in value and value["structured_response"]:
                print("Structured Output:", value["structured_response"].model_dump_json(indent=2))

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()

while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except Exception as e:
        print(e)
        break

User:  Vladimir Nabokov


Assistant: Structured Response: {
  "achievement_evidence": "1. Evidence of a one-time achievement",
  "achievement_evidence_url": "https://en.wikipedia.org/wiki/Vladimir_Nabokov#Literary_career",
  "field_one": "a. Receipt of lesser nationally or internationally recognized prizes or awards for excellence in the field of endeavor",
  "field_one_url": "https://en.wikipedia.org/wiki/Vladimir_Nabokov#Awards_and_honors",
  "field_two": "c. Published material about the alien in professional or major trade publications or other major media",
  "field_two_url": "https://www.theparisreview.org/interviews/4310/the-art-of-fiction-no-40-vladimir-nabokov",
  "field_three": "e. Original scientific, scholarly, artistic, athletic, or business-related contributions of major significance in the field",
  "field_three_url": "https://www.britannica.com/biography/Vladimir-Nabokov"
}


User:  Vladimir Nabokov


Assistant: Structured Response: {
  "achievement_evidence": "Vladimir Nabokov (1899–1977) was a Russian-American novelist, poet, and entomologist, best known for his novel *Lolita* (1955).",
  "achievement_evidence_url": "https://www.britannica.com/biography/Vladimir-Nabokov",
  "field_one": "Literature/Writing",
  "field_one_url": "https://www.britannica.com/biography/Vladimir-Nabokov",
  "field_two": "Entomology",
  "field_two_url": "https://www.newyorker.com/books/page-turner/vladimir-nabokov-butterfly-hunter",
  "field_three": "Criticism/Translation",
  "field_three_url": "https://www.theparisreview.org/interviews/4310/the-art-of-fiction-no-40-vladimir-nabokov"
}


1. Evidence of a one-time achievement (for example, a major internationally recognized award); or
2. At least three of the following:
    1. Receipt of lesser nationally or internationally recognized prizes or awards for excellence in the field of endeavor;
    2. Membership in associations in the field, which require outstanding achievements as judged by recognized national or international experts;
    3. Published material about the alien in professional or major trade publications or other major media;
    4. Participation on a panel or individually as a judge of others’ work in the field or a related field;
    5. Original scientific, scholarly, artistic, athletic, or business-related contributions of major significance in the field;
    6. Authorship of scholarly articles in the field in professional or major trade publications or other major media;
    7. Display of the alien’s work at artistic exhibitions or showcases;
    8. Evidence that the alien has performed in a leading or critical role for organizations or establishments that have distinguished reputations;
    9. Evidence that the alien has commanded a high salary or other high compensation for services; and
    10. Evidence of commercial successes in the performing arts as shown by box office receipts or music or video
sales.

In [136]:
with open('fields.json', 'r') as f:
    data = json.load(f)

journals = data["journals"]
journals = [{"field": journal["field"], "domains": [domain.split("/")[0] for domain in journal["domains"]]} for journal in journals]
add = data["additional_general_academic_resources"]
add = [url.split("/")[0] for url in add]
new = data["general_news_sources"]
new = [url.split("/")[0] for url in new]

new_data = {"journals": journals, "additional_general_academic_resources": add, "general_news_sources": new}

with open('fields.json', 'w') as f:
    data = f.write(json.dumps(new_data, indent=4))